<a href="https://colab.research.google.com/github/zyh404/skin_lesion_analysis/blob/main/ISIC2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import shutil

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
shutil.copy2("drive/My Drive/skin lesion analysis/images_input.zip","./")

'./images_input.zip'

In [6]:
!unzip -q images_input.zip

In [7]:
shutil.copy2("drive/My Drive/skin lesion analysis/ISIC2018_Task3_Training_GroundTruth.csv","./")

'./ISIC2018_Task3_Training_GroundTruth.csv'

In [8]:
import os
import math
import pandas as pd
import numpy as np

In [9]:
np.random.seed(36)

In [10]:
df = pd.read_csv("./ISIC2018_Task3_Training_GroundTruth.csv")

In [11]:
for col in df.columns:
  if col != "image":
    s = df[col].sum()
    c = df[col].count()
    print("Column: {}\t Count: {}\t Percentage: {}".format(col, s, round((s/c)*100, 2)))

Column: MEL	 Count: 1113.0	 Percentage: 11.11
Column: NV	 Count: 6705.0	 Percentage: 66.95
Column: BCC	 Count: 514.0	 Percentage: 5.13
Column: AKIEC	 Count: 327.0	 Percentage: 3.27
Column: BKL	 Count: 1099.0	 Percentage: 10.97
Column: DF	 Count: 115.0	 Percentage: 1.15
Column: VASC	 Count: 142.0	 Percentage: 1.42


In [12]:
df["class"] = df.drop("image", axis=1).apply(lambda x: x.to_numpy().argmax(), axis=1)

In [13]:
a = list(df.columns)
for i in ["image", "class"]:
  a.remove(i)
a = dict(zip(a, range(len(a))))
print(a)

{'MEL': 0, 'NV': 1, 'BCC': 2, 'AKIEC': 3, 'BKL': 4, 'DF': 5, 'VASC': 6}


In [14]:
df.drop(a.keys(), axis=1, inplace=True)
df.to_csv("isic_labels.csv", index=False)

In [15]:
test_idx = []
for c in df["class"].unique():
    temp = df[df['class'].apply(lambda x: True if x == c else False)]
    test_idx += temp.sample(math.floor(temp.shape[0]/10)).index.to_list()

In [16]:
test_df = df.iloc[test_idx].reset_index(drop=True)

In [17]:
train_df = df.drop(labels=test_idx).reset_index(drop=True)

In [18]:
train_df.to_csv("isic_train_labels.csv", index=False)
test_df.to_csv("isic_test_labels.csv", index=False)

In [19]:
df = pd.read_csv("isic_train_labels.csv")

In [20]:
sum(df["class"] == 1)

6035

In [21]:
classes = df["class"].unique()
max_val = 6035

In [22]:
path = "images_input"
def cloning(img, count):
  """
  img: Image Name
  count: Number of times `img` needs to be copied.
  """
  images = []
  for i in range(count):
    ii = img+"_"+str(i)
    img_path = os.path.join(path, img+".jpg")
    ii_path = os.path.join(path, ii+".jpg")
    # Creating a copy of `img` in the file system
    shutil.copy(img_path, ii_path)
    images.append(ii)
  return images

In [23]:
images, labels = [], []
for c in classes:
    if c == 1:
        continue
    data = df[df["class"].apply(lambda x: True if x == c else False)]
    l = data.image.shape[0]
    rest = max_val - (math.floor(max_val/l)*l)
    ns = [j+1 if i < rest else j for i, j in enumerate([math.floor(max_val/l)]*l)]
    for i, n in zip(data.image.values, ns):
        images += cloning(i, n)
        labels += [c]*n

In [24]:
oversampled = pd.DataFrame({"image": images, "class": labels})
df_1 = df[df["class"].apply(lambda x: True if x == 1 else False)]
balanced = pd.concat([oversampled, df_1]).reset_index(drop=True)
balanced["is_valid"] = False
test_df["is_valid"] = True
balanced = pd.concat([balanced, test_df]).reset_index(drop=True)
balanced.to_csv("isic_labels_balanced.csv", index=False)

In [25]:
!curl https://course.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9127    0  9127    0     0  25005      0 --:--:-- --:--:-- --:--:-- 25005
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE html>'


In [26]:
from fastai.vision import *

In [27]:
tfms = get_transforms(max_rotate=20, max_zoom=1.5, max_lighting=0.5, max_warp=0.4, p_affine=1., p_lighting=1., flip_vert=True)


In [28]:
src = (ImageList.from_csv("./", "isic_labels_balanced.csv", folder="images_input", suffix=".jpg")
                .split_from_df() # uses `is_valid` column
                .label_from_df())


In [ ]:
data = (src.transform(tfms, size=224)
           .databunch()
           .normalize(imagenet_stats))

In [28]:
learn = cnn_learner(data, models.resnet34, metrics=Recall(average="macro"))

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

In [ ]:
learn.fit_one_cycle(4)

In [30]:
learn.save('stage1-sz224-rn34')

In [31]:
shutil.copy2("./models/stage1-sz224-rn34.pth", "./drive/My Drive/skin lesion analysis")

'./drive/My Drive/skin lesion analysis/stage1-sz224-rn34.pth'

In [43]:
shutil.copy2("./drive/My Drive/skin lesion analysis/stage1-sz224-rn34.pth","./models")

'./models'

In [45]:
learn=create_cnn(data, models.resnet34, metrics=error_rate)

/usr/local/lib/python3.7/dist-packages/fastai/vision/learner.py:109: UserWarning: `create_cnn` is deprecated and is now named `cnn_learner`.
  warn("`create_cnn` is deprecated and is now named `cnn_learner`.")


In [48]:
learn.path = Path("./drive/My Drive/skin lesion analysis")

In [ ]:
learn.load("stage1-sz224-rn34")

In [54]:
learn.unfreeze()
learn.fit_one_cycle(3, slice(1e-6, 1e-4))

epoch,train_loss,valid_loss,error_rate,time
0,0.535938,0.621609,0.256513,5:56:24
1,0.473286,0.586881,0.239479,6:03:48
2,0.444768,0.567329,0.239479,5:40:20


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3680: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  "The default behavior for interpolate/upsample with float scale_factor changed "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3680: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  "The default